In [1]:
import cython
import numpy as np

%load_ext cython

In [16]:
np.__version__

'1.24.3'

In [2]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np

cdef class ListOfNdarrays:
    
    cdef Py_ssize_t y_size, index
    cdef list storage
    cdef const double[::1] y0_view
    cdef public double[:, ::1] solution
    
    def __init__(self, const double[::1] y0):
        
        self.y0_view = y0
        self.y_size = self.y0_view.size
        self.storage = list()
        
        self.run()
        
    cdef run(self) noexcept:
        
        cdef Py_ssize_t i, j 
        cdef double[::1] y_view
        y = np.empty(self.y_size, dtype=np.float64, order='C')
        y_view = y
        
        while True:
            
            # For testing purposes, let the loop run for a set amount of steps.
            # In reality, _we won't know how many steps will be required_.
            # This value is based on the calculations performed below.
            if self.index > 10_000:
                break
            
            for i in range(self.y_size):
                # Perform some calcualtion intensive work.
                y_view[i] = self.y0_view[i] + (<double>self.index / (<double>i + 1.))
            
            # Make a copy of y. Store it in the list.
            self.storage.append(y_view.copy())
            self.index += 1
        
        # Save results in a more user friendly format
        solution_array = np.empty((self.y_size, self.index), dtype=np.float64, order='C')
        self.solution = solution_array
        for i in range(self.y_size):
            for j in range(self.index):
                self.solution[i, j] = self.storage[j][i]

Content of stdout:
_cython_magic_d93813a0aad0792d24d97d9ce21c6a7adaf271a0.c
C:\Users\joepr\.ipython\cython\_cython_magic_d93813a0aad0792d24d97d9ce21c6a7adaf271a0.c(1467): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_d93813a0aad0792d24d97d9ce21c6a7adaf271a0.c(1418): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_d93813a0aad0792d24d97d9ce21c6a7adaf271a0.cp311-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_d93813a0aad0792d24d97d9ce21c6a7adaf271a0.cp311-win_amd64.exp
Generating code
Finished generating code

In [3]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np

cdef class ManuallyGrowNdarray:
    
    cdef Py_ssize_t y_size, index, expected_size
    cdef public Py_ssize_t concats
    cdef double[:, ::1] storage
    cdef const double[::1] y0_view
    cdef public double[:, ::1] solution
    
    def __init__(self, const double[::1] y0, Py_ssize_t expected_size):
        
        self.y0_view = y0
        self.y_size = self.y0_view.size
        
        # Setup a storage array based on a user-defined guess on the final size.
        self.concats = 1
        self.expected_size = expected_size
        storage_arr = np.empty((self.y_size, self.expected_size), dtype=np.float64, order='C')
        self.storage = storage_arr
        
        self.run()
        
    cdef run(self) noexcept:
        
        cdef Py_ssize_t i, j
        cdef double[::1] y_view
        cdef double[:, ::1] new_storage_view
        y = np.empty(self.y_size, dtype=np.float64, order='C')
        y_view = y
        
        while True:
            
            # For testing purposes, let the loop run for a set amount of steps.
            # In reality, _we won't know how many steps will be required_.
            # This value is based on the calculations performed below.
            if self.index > 10_000:
                break
            
            for i in range(self.y_size):
                # Perform some calcualtion intensive work.
                y_view[i] = self.y0_view[i] + (<double>self.index / (<double>i + 1.))
            
             # Check if our array is large enough
            if self.index >= (self.concats * self.expected_size):
                # We need to make a larger array
                self.concats += 1
                new_storage = np.empty((self.y_size, self.expected_size * self.concats), dtype=np.float64, order='C')
                new_storage_view = new_storage
                
                # Populate old values
                for i in range(self.y_size):
                    for j in range(self.index):
                        new_storage_view[i, j] = self.storage[i, j]
                # Reassign variable
                self.storage = new_storage_view
            
            # Add results to storage
            for i in range(self.y_size):
                self.storage[i, self.index] = y_view[i]
            
            self.index += 1
        
        # Remove any extra values not used at end of array
        solution_array = np.empty((self.y_size, self.index), dtype=np.float64, order='C')
        self.solution = solution_array
        for i in range(self.y_size):
            for j in range(self.index):
                self.solution[i, j] = self.storage[i, j]
        

Content of stdout:
_cython_magic_9a6ab33c6f3f628bddd40a30d56e3b2a0858217c.c
C:\Users\joepr\.ipython\cython\_cython_magic_9a6ab33c6f3f628bddd40a30d56e3b2a0858217c.c(1468): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_9a6ab33c6f3f628bddd40a30d56e3b2a0858217c.c(1419): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_9a6ab33c6f3f628bddd40a30d56e3b2a0858217c.cp311-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_9a6ab33c6f3f628bddd40a30d56e3b2a0858217c.cp311-win_amd64.exp
Generating code
Finished generating code

In [4]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np

cdef class NpResize:
    
    cdef Py_ssize_t y_size, index, expected_size
    cdef public Py_ssize_t concats
    cdef double[:, ::1] storage
    cdef const double[::1] y0_view
    cdef public double[:, ::1] solution
    
    def __init__(self, const double[::1] y0, Py_ssize_t expected_size):
        
        self.y0_view = y0
        self.y_size = self.y0_view.size
        
        # Setup a storage array based on a user-defined guess on the final size.
        self.concats = 1
        self.expected_size = expected_size
        storage_arr = np.empty((self.y_size, self.expected_size), dtype=np.float64, order='C')
        self.storage = storage_arr
        
        self.run()
        
    cdef run(self) noexcept:
        
        cdef Py_ssize_t i, j
        cdef double[::1] y_view
        y = np.empty(self.y_size, dtype=np.float64, order='C')
        y_view = y
        
        while True:
            
            # For testing purposes, let the loop run for a set amount of steps.
            # In reality, _we won't know how many steps will be required_.
            # This value is based on the calculations performed below.
            if self.index > 10_000:
                break
            
            for i in range(self.y_size):
                # Perform some calcualtion intensive work.
                y_view[i] = self.y0_view[i] + (<double>self.index / (<double>i + 1.))
            
             # Check if our array is large enough
            if self.index >= (self.concats * self.expected_size):
                # We need to make a larger array
                self.concats += 1
                self.storage = np.resize(self.storage, (self.y_size, self.concats * self.expected_size))
            
            # Add results to storage
            for i in range(self.y_size):
                self.storage[i, self.index] = y_view[i]
            
            self.index += 1
        
        # Remove any extra values not used at end of array
        solution_array = np.empty((self.y_size, self.index), dtype=np.float64, order='C')
        self.solution = solution_array
        for i in range(self.y_size):
            for j in range(self.index):
                self.solution[i, j] = self.storage[i, j]
        

Content of stdout:
_cython_magic_9b0a735f0f3e54bbb469cdd036fd895b32e68323.c
C:\Users\joepr\.ipython\cython\_cython_magic_9b0a735f0f3e54bbb469cdd036fd895b32e68323.c(1469): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_9b0a735f0f3e54bbb469cdd036fd895b32e68323.c(1420): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_9b0a735f0f3e54bbb469cdd036fd895b32e68323.cp311-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_9b0a735f0f3e54bbb469cdd036fd895b32e68323.cp311-win_amd64.exp
Generating code
Finished generating code

In [5]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

import numpy as np

cdef class GrowCArray:
    
    cdef Py_ssize_t y_size, index, expected_size, new_size
    cdef public Py_ssize_t concats
    cdef double* storage
    cdef const double[::1] y0_view
    cdef public double[:, ::1] solution
    
    def __init__(self, const double[::1] y0, Py_ssize_t expected_size):
        
        self.y0_view = y0
        self.y_size = self.y0_view.size
        
        # Setup a storage array based on a user-defined guess on the final size.
        self.concats = 1
        self.expected_size = expected_size
        self.new_size = self.expected_size
        self.storage = <double*>PyMem_Malloc(self.y_size * self.expected_size * sizeof(double))
        if not self.storage:
            raise MemoryError()
        
        self.run()
        
    cdef run(self) noexcept:
        
        cdef Py_ssize_t i, j
        cdef double[::1] y_view
        cdef double* new_storage
        y = np.empty(self.y_size, dtype=np.float64, order='C')
        y_view = y
        
        while True:
            
            # For testing purposes, let the loop run for a set amount of steps.
            # In reality, _we won't know how many steps will be required_.
            # This value is based on the calculations performed below.
            if self.index > 10_000:
                break
            
            for i in range(self.y_size):
                # Perform some calcualtion intensive work.
                y_view[i] = self.y0_view[i] + (<double>self.index / (<double>i + 1.))
            
             # Check if our array is large enough
            if self.index >= (self.concats * self.expected_size):
                # We need to make a larger array
                self.concats += 1
                self.new_size = self.concats * self.expected_size
                self.storage = <double*>PyMem_Realloc(self.storage, self.y_size * self.new_size * sizeof(double))
                if not self.storage:
                    raise MemoryError()
            
            # Add results to storage
            for i in range(self.y_size):
                self.storage[self.y_size * self.index + i] = y_view[i]
            
            self.index += 1
        
        # Remove any extra values not used at end of array
        solution_array = np.empty((self.y_size, self.index), dtype=np.float64, order='C')
        self.solution = solution_array
        for i in range(self.y_size):
            for j in range(self.index):
                self.solution[i, j] = self.storage[j * self.y_size + i]
                
    def __dealloc__(self):
        PyMem_Free(self.storage)
        

Content of stdout:
_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890cdbe.c
C:\Users\joepr\.ipython\cython\_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890cdbe.c(1471): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890cdbe.c(1422): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890cdbe.cp311-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890cdbe.cp311-win_amd64.exp
Generating code
Finished generating code

In [6]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

from libc.stdlib cimport malloc, free, realloc

import numpy as np

cdef class GrowCArrayCAllocs:
    
    cdef Py_ssize_t y_size, index, expected_size, new_size
    cdef public Py_ssize_t concats
    cdef double* storage
    cdef const double[::1] y0_view
    cdef public double[:, ::1] solution
    
    def __init__(self, const double[::1] y0, Py_ssize_t expected_size):
        
        self.y0_view = y0
        self.y_size = self.y0_view.size
        
        # Setup a storage array based on a user-defined guess on the final size.
        self.concats = 1
        self.expected_size = expected_size
        self.new_size = self.expected_size
        self.storage = <double*>malloc(self.y_size * self.expected_size * sizeof(double))
        if not self.storage:
            raise MemoryError()
        
        self.run()
        
    cdef run(self) noexcept:
        
        cdef Py_ssize_t i, j
        cdef double[::1] y_view
        cdef double* new_storage
        y = np.empty(self.y_size, dtype=np.float64, order='C')
        y_view = y
        
        while True:
            
            # For testing purposes, let the loop run for a set amount of steps.
            # In reality, _we won't know how many steps will be required_.
            # This value is based on the calculations performed below.
            if self.index > 10_000:
                break
            
            for i in range(self.y_size):
                # Perform some calcualtion intensive work.
                y_view[i] = self.y0_view[i] + (<double>self.index / (<double>i + 1.))
            
             # Check if our array is large enough
            if self.index >= (self.concats * self.expected_size):
                # We need to make a larger array
                self.concats += 1
                self.new_size = self.concats * self.expected_size
                self.storage = <double*>realloc(self.storage, self.y_size * self.new_size * sizeof(double))
                if not self.storage:
                    raise MemoryError()
            
            # Add results to storage
            for i in range(self.y_size):
                self.storage[self.y_size * self.index + i] = y_view[i]
            
            self.index += 1
        
        # Remove any extra values not used at end of array
        solution_array = np.empty((self.y_size, self.index), dtype=np.float64, order='C')
        self.solution = solution_array
        for i in range(self.y_size):
            for j in range(self.index):
                self.solution[i, j] = self.storage[j * self.y_size + i]
                
    def __dealloc__(self):
        free(self.storage)
        

Content of stdout:
_cython_magic_753a1fd0ac033bead0d9b871ec0a30e0d4903718.c
C:\Users\joepr\.ipython\cython\_cython_magic_753a1fd0ac033bead0d9b871ec0a30e0d4903718.c(1472): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_753a1fd0ac033bead0d9b871ec0a30e0d4903718.c(1423): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_753a1fd0ac033bead0d9b871ec0a30e0d4903718.cp311-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_753a1fd0ac033bead0d9b871ec0a30e0d4903718.cp311-win_amd64.exp
Generating code
Finished generating code

In [7]:
# 10 y values
y0 = np.random.random_sample(10)

In [8]:
# Python List of ndarrays
# Times: 25.2ms; 25.8ms
%timeit ListOfNdarrays(y0)

24.9 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
 ManuallyGrowNdarray(y0, 10_001).solution.shape

(10, 10001)

In [17]:
# Manually expanding numpy storage by fixed increments based on initial guess
# Times if we have a Perfect guess: 280us
%timeit ManuallyGrowNdarray(y0, 10_001)
# Times if we overshoot by 2x:  641us
%timeit ManuallyGrowNdarray(y0, 10 * 10_001)
# Times if we undershoot by 2x: 687us
%timeit ManuallyGrowNdarray(y0, int(0.1 * 10_001))

924 µs ± 7.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
648 µs ± 8.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
619 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
# Manually expanding numpy storage by fixed increments based on initial guess
# Times if we have a Perfect guess: 279us
%timeit NpResize(y0, 10_001)
# Times if we overshoot by 2x:  655us
%timeit NpResize(y0, 10 * 10_001)
# Times if we undershoot by 2x: 805us
%timeit NpResize(y0, int(0.1 * 10_001))

282 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
641 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
3.2 ms ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
# Use c pointers with C-Api Pymallocs
# Times if we have a Perfect guess: 296us
%timeit GrowCArray(y0, 10_001)
# Times if we overshoot by 2x: 472us
%timeit GrowCArray(y0, 10 * 10_001)
# Times if we undershoot by 2x: 574us
%timeit GrowCArray(y0, int(10))

296 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
483 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
562 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
GrowCArray(y0, int(10)).growths

AttributeError: '_cython_magic_6e42e98b8af4db6b3c48dc5a51a9d04a9890' object has no attribute 'growths'

In [13]:
# Use c pointers and c mallocs
# Times if we have a Perfect guess: 303us
%timeit GrowCArrayCAllocs(y0, 10_001)
# Times if we overshoot by 2x: 473us
%timeit GrowCArrayCAllocs(y0, 2 * 10_001)
# Times if we undershoot by 2x: 578us
%timeit GrowCArrayCAllocs(y0, int(0.5 * 10_001))

298 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
484 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
567 µs ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
